In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
import joblib
import warnings
warnings.filterwarnings('ignore')

# Paths
BASE_DIR = Path(r'c:\Users\Acer\Desktop\Forex-Signal-App')
DATA_DIR = BASE_DIR / 'data'
MODELS_DIR = BASE_DIR / 'models'

# Trading Parameters
INITIAL_BALANCE = 1000  # $1,000
RISK_PER_TRADE = 10     # $10 (1%)
TP_PIPS = 15
SL_PIPS = 10
PIP_VALUE = 0.0001

print("="*70)
print("📊 ALL MODELS BACKTEST COMPARISON")
print("="*70)
print(f"Initial Balance: ${INITIAL_BALANCE:,}")
print(f"Risk per Trade: ${RISK_PER_TRADE}")
print(f"TP: {TP_PIPS} pips, SL: {SL_PIPS} pips")

📊 ALL MODELS BACKTEST COMPARISON
Initial Balance: $1,000
Risk per Trade: $10
TP: 15 pips, SL: 10 pips


## 1. Load Test Data

In [3]:
# Load test data
test_df = pd.read_csv(DATA_DIR / 'EUR_USD_test.csv')
if 'timestamp' in test_df.columns:
    test_df.rename(columns={'timestamp': 'time'}, inplace=True)
test_df['time'] = pd.to_datetime(test_df['time'])

print(f"Test Data: {len(test_df):,} rows")
print(f"Period: {test_df['time'].min()} to {test_df['time'].max()}")

Test Data: 296,778 rows
Period: 2024-12-31 16:00:00+00:00 to 2025-10-17 06:11:00+00:00


## 2. Feature Engineering Functions

In [4]:
def add_features_v10(df):
    """V10 Feature Engineering"""
    df = df.copy()
    
    # Time Features
    df['hour'] = df['time'].dt.hour
    df['day_of_week'] = df['time'].dt.dayofweek
    df['is_london'] = ((df['hour'] >= 8) & (df['hour'] < 16)).astype(int)
    df['is_ny'] = ((df['hour'] >= 13) & (df['hour'] < 21)).astype(int)
    df['is_overlap'] = ((df['hour'] >= 13) & (df['hour'] < 16)).astype(int)
    
    # Moving Averages
    for p in [5, 10, 20, 50, 200]:
        df[f'sma_{p}'] = df['close'].rolling(p).mean()
        df[f'ema_{p}'] = df['close'].ewm(span=p, adjust=False).mean()
    
    # RSI
    delta = df['close'].diff()
    gain = delta.where(delta > 0, 0).rolling(14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(14).mean()
    rs = gain / (loss + 1e-10)
    df['rsi'] = 100 - (100 / (1 + rs))
    
    # MACD
    ema12 = df['close'].ewm(span=12).mean()
    ema26 = df['close'].ewm(span=26).mean()
    df['macd'] = ema12 - ema26
    df['macd_signal'] = df['macd'].ewm(span=9).mean()
    df['macd_hist'] = df['macd'] - df['macd_signal']
    
    # Bollinger Bands
    df['bb_mid'] = df['close'].rolling(20).mean()
    df['bb_std'] = df['close'].rolling(20).std()
    df['bb_upper'] = df['bb_mid'] + 2 * df['bb_std']
    df['bb_lower'] = df['bb_mid'] - 2 * df['bb_std']
    df['bb_width'] = (df['bb_upper'] - df['bb_lower']) / (df['bb_mid'] + 1e-10)
    
    # ATR
    df['tr0'] = abs(df['high'] - df['low'])
    df['tr1'] = abs(df['high'] - df['close'].shift())
    df['tr2'] = abs(df['low'] - df['close'].shift())
    df['tr'] = df[['tr0', 'tr1', 'tr2']].max(axis=1)
    df['atr'] = df['tr'].rolling(14).mean()
    
    # Stochastic
    low_14 = df['low'].rolling(14).min()
    high_14 = df['high'].rolling(14).max()
    df['stoch_k'] = 100 * (df['close'] - low_14) / (high_14 - low_14 + 1e-10)
    df['stoch_d'] = df['stoch_k'].rolling(3).mean()
    
    # Price patterns
    df['body'] = df['close'] - df['open']
    df['body_pct'] = df['body'] / (df['open'] + 1e-10)
    df['upper_wick'] = df['high'] - df[['open', 'close']].max(axis=1)
    df['lower_wick'] = df[['open', 'close']].min(axis=1) - df['low']
    df['range'] = df['high'] - df['low']
    
    # Momentum
    for p in [5, 10, 20]:
        df[f'momentum_{p}'] = df['close'] - df['close'].shift(p)
        df[f'roc_{p}'] = df['close'].pct_change(p) * 100
    
    # Trend strength
    df['trend_5'] = (df['close'] > df['sma_5']).astype(int)
    df['trend_20'] = (df['close'] > df['sma_20']).astype(int)
    df['trend_50'] = (df['close'] > df['sma_50']).astype(int)
    df['trend_strength'] = df['trend_5'] + df['trend_20'] + df['trend_50']
    
    # Volatility
    df['volatility_10'] = df['close'].rolling(10).std()
    df['volatility_20'] = df['close'].rolling(20).std()
    
    # Volume
    if 'volume' in df.columns:
        df['volume_sma'] = df['volume'].rolling(20).mean()
        df['volume_ratio'] = df['volume'] / (df['volume_sma'] + 1e-10)
    
    # Drop temp columns
    df.drop(['tr0', 'tr1', 'tr2', 'tr', 'bb_std'], axis=1, inplace=True, errors='ignore')
    
    return df.dropna()

# V11 uses same features as V10
add_features_v11 = add_features_v10

def add_features_v12(df):
    """V12 Feature Engineering (same as V10/V11 for consistency)"""
    return add_features_v10(df)

print("✓ Feature engineering functions defined")

✓ Feature engineering functions defined


## 3. Backtest Function

In [19]:
def run_backtest(signals, prices_df, initial_balance=1000, tp_pips=15, sl_pips=10):
    """
    Run backtest simulation
    
    signals: array of (index, signal_type, confidence)
        signal_type: 1=BUY, -1=SELL, 0=NO_TRADE
    """
    balance = initial_balance
    trades = []
    wins = 0
    losses = 0
    
    # Convert pips to price
    tp_price = tp_pips * 0.0001
    sl_price = sl_pips * 0.0001
    
    # Risk per trade
    risk = 10  # $10 per trade
    
    for idx, signal_type, confidence in signals:
        if signal_type == 0:
            continue
            
        if idx + 60 >= len(prices_df):
            continue
            
        entry_price = prices_df.iloc[idx]['close']
        
        # Look forward for TP/SL hit
        future_data = prices_df.iloc[idx+1:idx+61]
        
        if signal_type == 1:  # BUY
            tp_level = entry_price + tp_price
            sl_level = entry_price - sl_price
            
            tp_hit = (future_data['high'] >= tp_level).any()
            sl_hit = (future_data['low'] <= sl_level).any()
            
            if tp_hit and sl_hit:
                # Check which hit first
                tp_idx = future_data[future_data['high'] >= tp_level].index[0] if tp_hit else float('inf')
                sl_idx = future_data[future_data['low'] <= sl_level].index[0] if sl_hit else float('inf')
                if tp_idx < sl_idx:
                    profit = tp_pips  # Won
                    wins += 1
                else:
                    profit = -sl_pips  # Lost
                    losses += 1
            elif tp_hit:
                profit = tp_pips
                wins += 1
            elif sl_hit:
                profit = -sl_pips
                losses += 1
            else:
                # No TP/SL hit, close at market
                exit_price = future_data.iloc[-1]['close']
                profit = (exit_price - entry_price) / 0.0001
                if profit > 0:
                    wins += 1
                else:
                    losses += 1
                    
        else:  # SELL
            tp_level = entry_price - tp_price
            sl_level = entry_price + sl_price
            
            tp_hit = (future_data['low'] <= tp_level).any()
            sl_hit = (future_data['high'] >= sl_level).any()
            
            if tp_hit and sl_hit:
                tp_idx = future_data[future_data['low'] <= tp_level].index[0] if tp_hit else float('inf')
                sl_idx = future_data[future_data['high'] >= sl_level].index[0] if sl_hit else float('inf')
                if tp_idx < sl_idx:
                    profit = tp_pips
                    wins += 1
                else:
                    profit = -sl_pips
                    losses += 1
            elif tp_hit:
                profit = tp_pips
                wins += 1
            elif sl_hit:
                profit = -sl_pips
                losses += 1
            else:
                exit_price = future_data.iloc[-1]['close']
                profit = (entry_price - exit_price) / 0.0001
                if profit > 0:
                    wins += 1
                else:
                    losses += 1
        
        # Update balance ($1 per pip for 0.1 lot)
        balance += profit
        trades.append({
            'idx': idx,
            'type': 'BUY' if signal_type == 1 else 'SELL',
            'confidence': confidence,
            'profit': profit,
            'balance': balance
        })
    
    total_trades = wins + losses
    win_rate = wins / max(1, total_trades) * 100
    pf = (wins * tp_pips) / max(1, losses * sl_pips)
    
    return {
        'final_balance': balance,
        'profit': balance - initial_balance,
        'total_trades': total_trades,
        'wins': wins,
        'losses': losses,
        'win_rate': win_rate,
        'profit_factor': pf,
        'trades': trades
    }

print("✓ Backtest function defined")

✓ Backtest function defined


## 4. Test V10 Model

In [10]:
print("="*70)
print("📊 V10 MODEL BACKTEST (BUY-only)")
print("="*70)

# Load V10 models
v10_dir = MODELS_DIR / 'signal_generator_v10'
v10_scaler = joblib.load(v10_dir / 'scaler_v10.joblib')
v10_features = joblib.load(v10_dir / 'feature_cols_v10.joblib')
v10_weights = joblib.load(v10_dir / 'weights_v10.joblib')

v10_models = {}
for f in v10_dir.glob('*.joblib'):
    if 'scaler' not in f.stem and 'feature' not in f.stem and 'weights' not in f.stem and 'config' not in f.stem:
        name = f.stem.replace('_v10', '')
        v10_models[name] = joblib.load(f)

print(f"Loaded {len(v10_models)} V10 models: {list(v10_models.keys())}")

# Prepare test data with V10 features
v10_test = add_features_v10(test_df.copy())

# Ensure all required features exist
missing_features = [f for f in v10_features if f not in v10_test.columns]
for f in missing_features:
    v10_test[f] = 0

X_v10 = v10_test[v10_features].values
X_v10_scaled = v10_scaler.transform(X_v10)

# Get individual model predictions and probabilities
v10_proba_dict = {}
v10_preds_dict = {}
for name, model in v10_models.items():
    try:
        v10_proba_dict[name] = model.predict_proba(X_v10_scaled)
        v10_preds_dict[name] = model.predict(X_v10_scaled)
    except Exception as e:
        print(f"  Warning: {name} failed - {e}")

# Weighted ensemble probability
v10_final_proba = np.zeros((len(X_v10), 2))
for name, proba in v10_proba_dict.items():
    weight = v10_weights.get(name, 1/len(v10_models))
    v10_final_proba += weight * proba

# BUY probability (class 1)
buy_prob = v10_final_proba[:, 1] * 100

# V10 Agreement Bonus (7 models)
all_preds = np.array([v10_preds_dict[name] for name in v10_models.keys()])
buy_votes = np.sum(all_preds == 1, axis=0)

all_agree_buy = buy_votes == len(v10_models)  # All 7 agree
strong_buy = buy_votes >= 6  # 6+ agree
majority_buy = buy_votes >= 5  # 5+ agree

# Apply agreement bonus
confidence = buy_prob.copy()
confidence[all_agree_buy] = np.minimum(confidence[all_agree_buy] + 7, 100)
confidence[strong_buy & ~all_agree_buy] = np.minimum(confidence[strong_buy & ~all_agree_buy] + 4, 100)
confidence[majority_buy & ~strong_buy] = np.minimum(confidence[majority_buy & ~strong_buy] + 2, 100)

print(f"\nConfidence distribution:")
print(f"  >= 90%: {(confidence >= 90).sum():,}")
print(f"  >= 85%: {(confidence >= 85).sum():,}")
print(f"  >= 80%: {(confidence >= 80).sum():,}")
print(f"  >= 75%: {(confidence >= 75).sum():,}")
print(f"  >= 70%: {(confidence >= 70).sum():,}")

# Test at different thresholds - BUY ONLY
print("\n📈 V10 Results by Threshold (BUY-only):")
print("-"*70)

v10_results = {}
for threshold in [50, 60, 70, 75, 80, 85, 90]:
    signals = []
    for i, (idx, row) in enumerate(v10_test.iterrows()):
        if confidence[i] >= threshold:
            signals.append((idx, 1, confidence[i]))  # BUY signal only
    
    if len(signals) > 0:
        result = run_backtest(signals, test_df, INITIAL_BALANCE, TP_PIPS, SL_PIPS)
        v10_results[threshold/100] = result
        print(f"  {threshold}%+: Trades={result['total_trades']:,}, Win={result['wins']}, "
              f"WinRate={result['win_rate']:.1f}%, PF={result['profit_factor']:.2f}, "
              f"Final=${result['final_balance']:.2f}")
    else:
        print(f"  {threshold}%+: No signals")

📊 V10 MODEL BACKTEST (BUY-only)
Loaded 7 V10 models: ['cat1', 'cat2', 'lgb1', 'lgb2', 'xgb1', 'xgb2', 'xgb3']

Confidence distribution:
  >= 90%: 0
  >= 85%: 38
  >= 80%: 1,082
  >= 75%: 6,427
  >= 70%: 18,790

📈 V10 Results by Threshold (BUY-only):
----------------------------------------------------------------------

Confidence distribution:
  >= 90%: 0
  >= 85%: 38
  >= 80%: 1,082
  >= 75%: 6,427
  >= 70%: 18,790

📈 V10 Results by Threshold (BUY-only):
----------------------------------------------------------------------
  50%+: Trades=97,312, Win=46841, WinRate=48.1%, PF=1.39, Final=$22106.60
  60%+: Trades=48,666, Win=23594, WinRate=48.5%, PF=1.41, Final=$11401.30
  70%+: Trades=18,790, Win=9550, WinRate=50.8%, PF=1.55, Final=$9974.70
  75%+: Trades=6,427, Win=3357, WinRate=52.2%, PF=1.64, Final=$4846.50
  80%+: Trades=1,082, Win=409, WinRate=37.8%, PF=0.91, Final=$117.80
  85%+: Trades=38, Win=11, WinRate=28.9%, PF=0.61, Final=$868.40
  90%+: No signals


## 5. Test V11 Model (Binary BUY/SELL)

In [11]:
print("="*70)
print("📊 V11 MODEL BACKTEST (BUY-only)")
print("="*70)

# Load V11 models
v11_dir = MODELS_DIR / 'signal_generator_v11'
v11_scaler = joblib.load(v11_dir / 'scaler_v11.joblib')
v11_features = joblib.load(v11_dir / 'feature_cols_v11.joblib')
v11_weights = joblib.load(v11_dir / 'weights_v11.joblib')

v11_models = {}
for f in v11_dir.glob('*_binary.joblib'):
    name = f.stem.replace('_v11_binary', '')
    v11_models[name] = joblib.load(f)

print(f"Loaded {len(v11_models)} V11 binary models: {list(v11_models.keys())}")

# Prepare test data
v11_test = add_features_v11(test_df.copy())

missing_features = [f for f in v11_features if f not in v11_test.columns]
for f in missing_features:
    v11_test[f] = 0

X_v11 = v11_test[v11_features].values
X_v11_scaled = v11_scaler.transform(X_v11)

# Get individual model predictions and probabilities
v11_proba_dict = {}
v11_preds_dict = {}
for name, model in v11_models.items():
    try:
        v11_proba_dict[name] = model.predict_proba(X_v11_scaled)
        v11_preds_dict[name] = model.predict(X_v11_scaled)
    except Exception as e:
        print(f"  Warning: {name} failed - {e}")

# Weighted ensemble probability
v11_final_proba = np.zeros((len(X_v11), 2))
for name, proba in v11_proba_dict.items():
    weight = v11_weights.get(name, 1/len(v11_models))
    v11_final_proba += weight * proba

# BUY probability (class 1)
buy_prob_v11 = v11_final_proba[:, 1] * 100

# V11 Agreement Bonus (7 models)
all_preds_v11 = np.array([v11_preds_dict[name] for name in v11_models.keys()])
buy_votes_v11 = np.sum(all_preds_v11 == 1, axis=0)

all_agree_buy_v11 = buy_votes_v11 == len(v11_models)
strong_buy_v11 = buy_votes_v11 >= 6
majority_buy_v11 = buy_votes_v11 >= 5

# Apply agreement bonus
confidence_v11 = buy_prob_v11.copy()
confidence_v11[all_agree_buy_v11] = np.minimum(confidence_v11[all_agree_buy_v11] + 7, 100)
confidence_v11[strong_buy_v11 & ~all_agree_buy_v11] = np.minimum(confidence_v11[strong_buy_v11 & ~all_agree_buy_v11] + 4, 100)
confidence_v11[majority_buy_v11 & ~strong_buy_v11] = np.minimum(confidence_v11[majority_buy_v11 & ~strong_buy_v11] + 2, 100)

print(f"\nConfidence distribution:")
print(f"  >= 90%: {(confidence_v11 >= 90).sum():,}")
print(f"  >= 85%: {(confidence_v11 >= 85).sum():,}")
print(f"  >= 80%: {(confidence_v11 >= 80).sum():,}")
print(f"  >= 75%: {(confidence_v11 >= 75).sum():,}")
print(f"  >= 70%: {(confidence_v11 >= 70).sum():,}")

# Test at different thresholds - BUY ONLY
print("\n📈 V11 Results by Threshold (BUY-only):")
print("-"*70)

v11_results = {}
for threshold in [50, 60, 70, 75, 80, 85, 90]:
    signals = []
    for i, (idx, row) in enumerate(v11_test.iterrows()):
        if confidence_v11[i] >= threshold:
            signals.append((idx, 1, confidence_v11[i]))  # BUY signal only
    
    if len(signals) > 0:
        result = run_backtest(signals, test_df, INITIAL_BALANCE, TP_PIPS, SL_PIPS)
        v11_results[threshold/100] = result
        print(f"  {threshold}%+: Trades={result['total_trades']:,}, Win={result['wins']}, "
              f"WinRate={result['win_rate']:.1f}%, PF={result['profit_factor']:.2f}, "
              f"Final=${result['final_balance']:.2f}")
    else:
        print(f"  {threshold}%+: No signals")

📊 V11 MODEL BACKTEST (BUY-only)
Loaded 7 V11 binary models: ['cat1', 'cat2', 'lgb1', 'lgb2', 'xgb1', 'xgb2', 'xgb3']

Confidence distribution:
  >= 90%: 0
  >= 85%: 0
  >= 80%: 258
  >= 75%: 2,104
  >= 70%: 6,139

📈 V11 Results by Threshold (BUY-only):
----------------------------------------------------------------------

Confidence distribution:
  >= 90%: 0
  >= 85%: 0
  >= 80%: 258
  >= 75%: 2,104
  >= 70%: 6,139

📈 V11 Results by Threshold (BUY-only):
----------------------------------------------------------------------
  50%+: Trades=77,988, Win=37554, WinRate=48.2%, PF=1.39, Final=$13096.40
  50%+: Trades=77,988, Win=37554, WinRate=48.2%, PF=1.39, Final=$13096.40
  60%+: Trades=32,468, Win=15378, WinRate=47.4%, PF=1.35, Final=$1838.40
  60%+: Trades=32,468, Win=15378, WinRate=47.4%, PF=1.35, Final=$1838.40
  70%+: Trades=6,139, Win=3167, WinRate=51.6%, PF=1.60, Final=$1575.20
  70%+: Trades=6,139, Win=3167, WinRate=51.6%, PF=1.60, Final=$1575.20
  75%+: Trades=2,104, Win=1079, W

## 6. Test V12 Model (BUY-only)

In [12]:
print("="*70)
print("📊 V12 MODEL BACKTEST (BUY-only)")
print("="*70)

# Load V12 models
v12_dir = MODELS_DIR / 'signal_generator_v12'
v12_scaler = joblib.load(v12_dir / 'scaler_v12.joblib')
v12_features = joblib.load(v12_dir / 'feature_cols_v12.joblib')
v12_weights = joblib.load(v12_dir / 'model_weights_v12.joblib')

v12_models = {}
for f in v12_dir.glob('*.joblib'):
    if 'scaler' not in f.stem and 'feature' not in f.stem and 'weights' not in f.stem and 'config' not in f.stem:
        name = f.stem.replace('_v12', '')
        v12_models[name] = joblib.load(f)

print(f"Loaded {len(v12_models)} V12 models: {list(v12_models.keys())}")

# Prepare test data
v12_test = add_features_v12(test_df.copy())

missing_features = [f for f in v12_features if f not in v12_test.columns]
for f in missing_features:
    v12_test[f] = 0

X_v12 = v12_test[v12_features].values
X_v12_scaled = v12_scaler.transform(X_v12)

# Get ensemble predictions
v12_proba = np.zeros(len(X_v12))
for name, model in v12_models.items():
    try:
        proba = model.predict_proba(X_v12_scaled)[:, 1]
        weight = v12_weights.get(name, 1/len(v12_models))
        v12_proba += proba * weight
    except Exception as e:
        print(f"  Warning: {name} failed - {e}")

# Test at different thresholds
print("\n📈 V12 Results by Threshold:")
print("-"*70)

v12_results = {}
for threshold in [0.5, 0.6, 0.7, 0.75, 0.8, 0.85, 0.9]:
    signals = []
    for i, (idx, row) in enumerate(v12_test.iterrows()):
        if v12_proba[i] >= threshold:
            signals.append((idx, 1, v12_proba[i]))  # BUY signal
    
    if len(signals) > 0:
        result = run_backtest(signals, test_df, INITIAL_BALANCE, TP_PIPS, SL_PIPS)
        v12_results[threshold] = result
        print(f"  {threshold:.0%}: Trades={result['total_trades']:,}, Win={result['wins']}, "
              f"WinRate={result['win_rate']:.1f}%, PF={result['profit_factor']:.2f}, "
              f"Final=${result['final_balance']:.2f}")
    else:
        print(f"  {threshold:.0%}: No signals")

📊 V12 MODEL BACKTEST (BUY-only)
Loaded 9 V12 models: ['et', 'hgb', 'lgb1', 'lgb2', 'lgb3', 'rf', 'xgb1', 'xgb2', 'xgb3']
Loaded 9 V12 models: ['et', 'hgb', 'lgb1', 'lgb2', 'lgb3', 'rf', 'xgb1', 'xgb2', 'xgb3']

📈 V12 Results by Threshold:
----------------------------------------------------------------------

📈 V12 Results by Threshold:
----------------------------------------------------------------------
  50%: No signals
  50%: No signals
  60%: No signals
  60%: No signals
  70%: No signals
  70%: No signals
  75%: No signals
  75%: No signals
  80%: No signals
  80%: No signals
  85%: No signals
  85%: No signals
  90%: No signals
  90%: No signals


## 7. Final Comparison Summary

In [13]:
print("="*80)
print("🏆 FINAL MODEL COMPARISON SUMMARY")
print("="*80)
print(f"\nInitial Balance: ${INITIAL_BALANCE:,}")
print(f"Test Period: {test_df['time'].min().strftime('%Y-%m-%d')} to {test_df['time'].max().strftime('%Y-%m-%d')}")
print(f"TP: {TP_PIPS} pips, SL: {SL_PIPS} pips")
print()

# Create summary table
summary_data = []

# Best V10 result
if v10_results:
    best_v10_th = max(v10_results.keys(), key=lambda x: v10_results[x]['profit_factor'])
    best_v10 = v10_results[best_v10_th]
    summary_data.append({
        'Model': f'V10 ({best_v10_th:.0%})',
        'Trades': best_v10['total_trades'],
        'Wins': best_v10['wins'],
        'Losses': best_v10['losses'],
        'Win Rate': f"{best_v10['win_rate']:.1f}%",
        'PF': f"{best_v10['profit_factor']:.2f}",
        'Final Balance': f"${best_v10['final_balance']:.2f}",
        'Profit': f"${best_v10['profit']:.2f}"
    })

# Best V11 result
if v11_results:
    best_v11_th = max(v11_results.keys(), key=lambda x: v11_results[x]['profit_factor'])
    best_v11 = v11_results[best_v11_th]
    summary_data.append({
        'Model': f'V11 ({best_v11_th:.0%})',
        'Trades': best_v11['total_trades'],
        'Wins': best_v11['wins'],
        'Losses': best_v11['losses'],
        'Win Rate': f"{best_v11['win_rate']:.1f}%",
        'PF': f"{best_v11['profit_factor']:.2f}",
        'Final Balance': f"${best_v11['final_balance']:.2f}",
        'Profit': f"${best_v11['profit']:.2f}"
    })

# Best V12 result
if v12_results:
    best_v12_th = max(v12_results.keys(), key=lambda x: v12_results[x]['profit_factor'])
    best_v12 = v12_results[best_v12_th]
    summary_data.append({
        'Model': f'V12 ({best_v12_th:.0%})',
        'Trades': best_v12['total_trades'],
        'Wins': best_v12['wins'],
        'Losses': best_v12['losses'],
        'Win Rate': f"{best_v12['win_rate']:.1f}%",
        'PF': f"{best_v12['profit_factor']:.2f}",
        'Final Balance': f"${best_v12['final_balance']:.2f}",
        'Profit': f"${best_v12['profit']:.2f}"
    })

summary_df = pd.DataFrame(summary_data)
print(summary_df.to_string(index=False))

print("\n" + "="*80)
print("🎯 RECOMMENDATION:")

# Find best model overall
all_results = []
if v10_results:
    for th, r in v10_results.items():
        all_results.append(('V10', th, r))
if v11_results:
    for th, r in v11_results.items():
        all_results.append(('V11', th, r))
if v12_results:
    for th, r in v12_results.items():
        all_results.append(('V12', th, r))

if all_results:
    # Sort by profit factor (min 50 trades)
    valid_results = [(m, t, r) for m, t, r in all_results if r['total_trades'] >= 50]
    if valid_results:
        best = max(valid_results, key=lambda x: x[2]['profit_factor'])
        print(f"   Best Model: {best[0]} at {best[1]:.0%} threshold")
        print(f"   Final Balance: ${best[2]['final_balance']:.2f}")
        print(f"   Profit: ${best[2]['profit']:.2f} ({best[2]['profit']/INITIAL_BALANCE*100:.1f}%)")
        print(f"   Win Rate: {best[2]['win_rate']:.1f}%")
        print(f"   Profit Factor: {best[2]['profit_factor']:.2f}")
print("="*80)

🏆 FINAL MODEL COMPARISON SUMMARY

Initial Balance: $1,000
Test Period: 2024-12-31 to 2025-10-17
TP: 15 pips, SL: 10 pips

    Model  Trades  Wins  Losses Win Rate   PF Final Balance   Profit
V10 (75%)    6427  3357    3070    52.2% 1.64      $4846.50 $3846.50
V11 (70%)    6139  3167    2972    51.6% 1.60      $1575.20  $575.20

🎯 RECOMMENDATION:
   Best Model: V10 at 75% threshold
   Final Balance: $4846.50
   Profit: $3846.50 (384.7%)
   Win Rate: 52.2%
   Profit Factor: 1.64


## 8. V10 Label-Based Accuracy Check (Training Logic)

In [14]:
# Check using the SAME labeling logic as training
# This shows what the model actually learned vs what backtest shows

print("="*70)
print("🔍 V10 LABEL-BASED ACCURACY (Same as Training)")
print("="*70)

def create_labels_check(df, forward_periods=60, min_pips=15, ratio=1.5):
    """Same labeling as V10 training"""
    df = df.copy()
    min_move = min_pips * 0.0001
    
    df['future_max'] = df['high'].rolling(forward_periods).max().shift(-forward_periods)
    df['future_min'] = df['low'].rolling(forward_periods).min().shift(-forward_periods)
    
    df['up_move'] = df['future_max'] - df['close']
    df['down_move'] = df['close'] - df['future_min']
    
    # BUY: up >= min_pips AND up > down * ratio
    buy_cond = (df['up_move'] >= min_move) & (df['up_move'] > df['down_move'] * ratio)
    # SELL: down >= min_pips AND down > up * ratio
    sell_cond = (df['down_move'] >= min_move) & (df['down_move'] > df['up_move'] * ratio)
    
    df['true_signal'] = 0  # HOLD
    df.loc[buy_cond, 'true_signal'] = 1  # BUY
    df.loc[sell_cond, 'true_signal'] = -1  # SELL (but we use 0 for binary)
    
    # For binary: BUY=1, SELL=0 (only where signal != HOLD)
    df['binary_label'] = df['true_signal'].apply(lambda x: 1 if x == 1 else (0 if x == -1 else -99))
    
    df.drop(['future_max', 'future_min', 'up_move', 'down_move'], axis=1, inplace=True)
    return df

# Add labels to test data
v10_test_labeled = create_labels_check(v10_test.copy())
v10_test_labeled = v10_test_labeled[v10_test_labeled['binary_label'] != -99]  # Only BUY/SELL

print(f"Test samples with BUY/SELL labels: {len(v10_test_labeled):,}")
print(f"  BUY labels: {(v10_test_labeled['binary_label']==1).sum():,}")
print(f"  SELL labels: {(v10_test_labeled['binary_label']==0).sum():,}")

# Get predictions for these samples
test_indices = v10_test_labeled.index
v10_test_idx_map = {idx: i for i, idx in enumerate(v10_test.index)}

# Get confidence for labeled samples
labeled_confidence = []
labeled_true = []
for idx in test_indices:
    if idx in v10_test_idx_map:
        i = v10_test_idx_map[idx]
        labeled_confidence.append(confidence[i])
        labeled_true.append(v10_test_labeled.loc[idx, 'binary_label'])

labeled_confidence = np.array(labeled_confidence)
labeled_true = np.array(labeled_true)

print("\n📊 Label-Based Accuracy (Training Definition):")
print("-"*70)
print(f"{'Threshold':>10} | {'Signals':>10} | {'Correct':>10} | {'Accuracy':>10}")
print("-"*70)

for threshold in [50, 60, 70, 75, 80, 85, 90]:
    mask = labeled_confidence >= threshold
    if mask.sum() > 0:
        signals = mask.sum()
        correct = (labeled_true[mask] == 1).sum()  # Predicted BUY, actually BUY
        accuracy = correct / signals * 100
        print(f"{threshold:>8}%+ | {signals:>10,} | {correct:>10,} | {accuracy:>9.1f}%")
    else:
        print(f"{threshold:>8}%+ | {'0':>10} | {'-':>10} | {'-':>10}")

🔍 V10 LABEL-BASED ACCURACY (Same as Training)
Test samples with BUY/SELL labels: 80,296
  BUY labels: 41,895
  SELL labels: 38,401

📊 Label-Based Accuracy (Training Definition):
----------------------------------------------------------------------
 Threshold |    Signals |    Correct |   Accuracy
----------------------------------------------------------------------
      50%+ |     25,608 |     13,848 |      54.1%
      60%+ |     10,759 |      6,040 |      56.1%
      70%+ |      3,528 |      2,045 |      58.0%
      75%+ |      1,161 |        630 |      54.3%
      80%+ |        200 |        111 |      55.5%
      85%+ |         16 |          2 |      12.5%
      90%+ |          0 |          - |          -


## 9. Overfit Detection: Train vs Test

In [15]:
# Load train data and check accuracy there too
print("="*70)
print("⚠️ OVERFIT DETECTION: Train vs Test Accuracy")
print("="*70)

# Load train data
train_df = pd.read_csv(DATA_DIR / 'EUR_USD_1min.csv')
if 'timestamp' in train_df.columns:
    train_df.rename(columns={'timestamp': 'time'}, inplace=True)
train_df['time'] = pd.to_datetime(train_df['time'])

print(f"Train Data: {len(train_df):,} rows")

# Add features
train_features = add_features_v10(train_df.copy())

# Add labels
train_labeled = create_labels_check(train_features.copy())
train_labeled = train_labeled[train_labeled['binary_label'] != -99]

print(f"Train samples with BUY/SELL: {len(train_labeled):,}")

# Get predictions on train data
missing_train = [f for f in v10_features if f not in train_labeled.columns]
for f in missing_train:
    train_labeled[f] = 0

X_train_check = train_labeled[v10_features].values
X_train_scaled = v10_scaler.transform(X_train_check)

# Get train predictions
train_proba_dict = {}
train_preds_dict = {}
for name, model in v10_models.items():
    train_proba_dict[name] = model.predict_proba(X_train_scaled)
    train_preds_dict[name] = model.predict(X_train_scaled)

# Weighted ensemble
train_final_proba = np.zeros((len(X_train_check), 2))
for name, proba in train_proba_dict.items():
    weight = v10_weights.get(name, 1/len(v10_models))
    train_final_proba += weight * proba

train_buy_prob = train_final_proba[:, 1] * 100

# Agreement bonus
train_all_preds = np.array([train_preds_dict[name] for name in v10_models.keys()])
train_buy_votes = np.sum(train_all_preds == 1, axis=0)

train_all_agree = train_buy_votes == len(v10_models)
train_strong = train_buy_votes >= 6
train_majority = train_buy_votes >= 5

train_confidence = train_buy_prob.copy()
train_confidence[train_all_agree] = np.minimum(train_confidence[train_all_agree] + 7, 100)
train_confidence[train_strong & ~train_all_agree] = np.minimum(train_confidence[train_strong & ~train_all_agree] + 4, 100)
train_confidence[train_majority & ~train_strong] = np.minimum(train_confidence[train_majority & ~train_strong] + 2, 100)

train_true_labels = train_labeled['binary_label'].values

print("\n📊 TRAIN vs TEST Accuracy Comparison:")
print("-"*80)
print(f"{'Threshold':>10} | {'Train Sig':>10} | {'Train Acc':>10} | {'Test Sig':>10} | {'Test Acc':>10} | {'Gap':>10}")
print("-"*80)

for threshold in [50, 60, 70, 75, 80, 85, 90]:
    # Train
    train_mask = train_confidence >= threshold
    train_sig = train_mask.sum()
    train_acc = (train_true_labels[train_mask] == 1).mean() * 100 if train_sig > 0 else 0
    
    # Test
    test_mask = labeled_confidence >= threshold
    test_sig = test_mask.sum()
    test_acc = (labeled_true[test_mask] == 1).mean() * 100 if test_sig > 0 else 0
    
    gap = train_acc - test_acc
    gap_str = f"{gap:+.1f}%" if train_sig > 0 and test_sig > 0 else "-"
    
    overfit = "⚠️ OVERFIT" if gap > 20 else ("⚡" if gap > 10 else "✓")
    
    print(f"{threshold:>8}%+ | {train_sig:>10,} | {train_acc:>9.1f}% | {test_sig:>10,} | {test_acc:>9.1f}% | {gap_str:>8} {overfit}")

print("\n" + "="*80)
print("💡 CONCLUSION:")
print("   If Train Acc >> Test Acc, the model is OVERFITTED!")
print("   Need to retrain with better regularization or cross-validation.")
print("="*80)

⚠️ OVERFIT DETECTION: Train vs Test Accuracy
Train Data: 1,859,492 rows
Train samples with BUY/SELL: 393,249

📊 TRAIN vs TEST Accuracy Comparison:
--------------------------------------------------------------------------------
 Threshold |  Train Sig |  Train Acc |   Test Sig |   Test Acc |        Gap
--------------------------------------------------------------------------------
      50%+ |    154,176 |      57.6% |     25,608 |      54.1% |    +3.5% ✓
      60%+ |     82,468 |      62.6% |     10,759 |      56.1% |    +6.5% ✓
      70%+ |     31,023 |      69.4% |      3,528 |      58.0% |   +11.5% ⚡
      75%+ |     13,174 |      77.9% |      1,161 |      54.3% |   +23.7% ⚠️ OVERFIT
      80%+ |      3,552 |      87.2% |        200 |      55.5% |   +31.7% ⚠️ OVERFIT
      85%+ |        377 |      95.5% |         16 |      12.5% |   +83.0% ⚠️ OVERFIT
      90%+ |          8 |     100.0% |          0 |       0.0% |        - ⚠️ OVERFIT

💡 CONCLUSION:
   If Train Acc >> Test Acc, the

## 🚀 V13: TP/SL Based Labeling (Best Model)

In [17]:
print("="*80)
print("🚀 V13: TP/SL Based Labeling Backtest")
print("   WIN = TP hit first, LOSE = SL hit first")
print("="*80)

def add_features_v13(df):
    """V13 Feature Engineering - Same as V13 training"""
    df = df.copy()
    
    # Time Features
    df['hour'] = df['time'].dt.hour
    df['day_of_week'] = df['time'].dt.dayofweek
    df['is_london'] = ((df['hour'] >= 8) & (df['hour'] < 16)).astype(int)
    df['is_ny'] = ((df['hour'] >= 13) & (df['hour'] < 21)).astype(int)
    df['is_overlap'] = ((df['hour'] >= 13) & (df['hour'] < 16)).astype(int)
    
    # Moving Averages
    for p in [5, 10, 20, 50, 200]:
        df[f'sma_{p}'] = df['close'].rolling(p).mean()
        df[f'ema_{p}'] = df['close'].ewm(span=p, adjust=False).mean()
    
    # RSI
    delta = df['close'].diff()
    gain = delta.where(delta > 0, 0).rolling(14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(14).mean()
    rs = gain / (loss + 1e-10)
    df['rsi'] = 100 - (100 / (1 + rs))
    
    # MACD
    ema12 = df['close'].ewm(span=12).mean()
    ema26 = df['close'].ewm(span=26).mean()
    df['macd'] = ema12 - ema26
    df['macd_signal'] = df['macd'].ewm(span=9).mean()
    df['macd_hist'] = df['macd'] - df['macd_signal']
    
    # Bollinger Bands
    df['bb_mid'] = df['close'].rolling(20).mean()
    bb_std = df['close'].rolling(20).std()
    df['bb_upper'] = df['bb_mid'] + 2 * bb_std
    df['bb_lower'] = df['bb_mid'] - 2 * bb_std
    df['bb_width'] = (df['bb_upper'] - df['bb_lower']) / (df['bb_mid'] + 1e-10)
    df['bb_position'] = (df['close'] - df['bb_lower']) / (df['bb_upper'] - df['bb_lower'] + 1e-10)
    
    # ATR
    tr = pd.concat([
        abs(df['high'] - df['low']),
        abs(df['high'] - df['close'].shift()),
        abs(df['low'] - df['close'].shift())
    ], axis=1).max(axis=1)
    df['atr'] = tr.rolling(14).mean()
    df['atr_ratio'] = df['atr'] / (df['close'] + 1e-10)
    
    # Stochastic
    low_14 = df['low'].rolling(14).min()
    high_14 = df['high'].rolling(14).max()
    df['stoch_k'] = 100 * (df['close'] - low_14) / (high_14 - low_14 + 1e-10)
    df['stoch_d'] = df['stoch_k'].rolling(3).mean()
    
    # Price patterns
    df['body'] = df['close'] - df['open']
    df['body_pct'] = df['body'] / (df['open'] + 1e-10)
    df['upper_wick'] = df['high'] - df[['open', 'close']].max(axis=1)
    df['lower_wick'] = df[['open', 'close']].min(axis=1) - df['low']
    df['range'] = df['high'] - df['low']
    df['range_atr'] = df['range'] / (df['atr'] + 1e-10)
    
    # Momentum
    for p in [5, 10, 20]:
        df[f'momentum_{p}'] = df['close'] - df['close'].shift(p)
        df[f'roc_{p}'] = df['close'].pct_change(p) * 100
    
    # Trend
    df['trend_5'] = (df['close'] > df['sma_5']).astype(int)
    df['trend_20'] = (df['close'] > df['sma_20']).astype(int)
    df['trend_50'] = (df['close'] > df['sma_50']).astype(int)
    df['trend_strength'] = df['trend_5'] + df['trend_20'] + df['trend_50']
    
    # MA distances
    df['dist_sma20'] = (df['close'] - df['sma_20']) / (df['sma_20'] + 1e-10) * 100
    df['dist_sma50'] = (df['close'] - df['sma_50']) / (df['sma_50'] + 1e-10) * 100
    
    # Volatility
    df['volatility_10'] = df['close'].rolling(10).std()
    df['volatility_20'] = df['close'].rolling(20).std()
    
    # Volume
    if 'volume' in df.columns:
        df['volume_sma'] = df['volume'].rolling(20).mean()
        df['volume_ratio'] = df['volume'] / (df['volume_sma'] + 1e-10)
    
    return df.dropna()

# Load V13 models
v13_dir = MODELS_DIR / 'signal_generator_v13'
v13_models = {}
v13_features = joblib.load(v13_dir / 'feature_cols_v13.joblib')
v13_scaler = joblib.load(v13_dir / 'scaler_v13.joblib')
v13_weights = joblib.load(v13_dir / 'weights_v13.joblib')
v13_config = joblib.load(v13_dir / 'config_v13.joblib')

# Load all V13 models
for f in v13_dir.glob('*_v13.joblib'):
    name = f.stem.replace('_v13', '')
    if name not in ['scaler', 'feature_cols', 'weights', 'config']:
        v13_models[name] = joblib.load(f)
        
print(f"✓ Loaded {len(v13_models)} V13 models: {list(v13_models.keys())}")
print(f"✓ Features: {len(v13_features)}")

# Create features for test data using V13 feature engineering
v13_test = add_features_v13(test_df.copy())

# Check features
missing = [f for f in v13_features if f not in v13_test.columns]
if missing:
    print(f"⚠️ Missing features: {missing[:5]}...")
else:
    print(f"✓ All features available")

# Prepare data
X_v13 = v13_test[v13_features].values
X_v13_scaled = v13_scaler.transform(X_v13)

# Get predictions
v13_proba_dict = {}
v13_preds_dict = {}
for name, model in v13_models.items():
    v13_proba_dict[name] = model.predict_proba(X_v13_scaled)
    v13_preds_dict[name] = model.predict(X_v13_scaled)

# Weighted ensemble
v13_final_proba = np.zeros((len(X_v13_scaled), 2))
for name, proba in v13_proba_dict.items():
    v13_final_proba += v13_weights[name] * proba

# WIN probability
win_prob_v13 = v13_final_proba[:, 1] * 100

# Agreement bonus
all_preds_v13 = np.array([v13_preds_dict[name] for name in v13_models.keys()])
win_votes_v13 = np.sum(all_preds_v13 == 1, axis=0)

all_agree_win = win_votes_v13 == len(v13_models)
strong_win = win_votes_v13 >= (len(v13_models) - 1)
majority_win = win_votes_v13 >= (len(v13_models) // 2 + 1)

confidence_v13 = win_prob_v13.copy()
confidence_v13[all_agree_win] = np.minimum(confidence_v13[all_agree_win] + 5, 100)
confidence_v13[strong_win & ~all_agree_win] = np.minimum(confidence_v13[strong_win & ~all_agree_win] + 3, 100)
confidence_v13[majority_win & ~strong_win] = np.minimum(confidence_v13[majority_win & ~strong_win] + 1, 100)

print(f"\n📊 Confidence Distribution:")
print(f"  >= 70%: {(confidence_v13 >= 70).sum():,}")
print(f"  >= 60%: {(confidence_v13 >= 60).sum():,}")
print(f"  >= 50%: {(confidence_v13 >= 50).sum():,}")

🚀 V13: TP/SL Based Labeling Backtest
   WIN = TP hit first, LOSE = SL hit first
✓ Loaded 6 V13 models: ['cat1', 'cat2', 'lgb1', 'lgb2', 'xgb1', 'xgb2']
✓ Features: 50
✓ All features available

📊 Confidence Distribution:
  >= 70%: 0
  >= 60%: 8,203
  >= 50%: 30,024


In [21]:
# V13 Backtest by Threshold
print("\n" + "="*80)
print("📈 V13 BACKTEST RESULTS (BUY-only)")
print("="*80)

v13_results = {}

# Create index mapping
v13_test_indices = v13_test.index.tolist()

for threshold in [50, 55, 60, 65, 70, 75]:
    mask = confidence_v13 >= threshold
    signals_count = mask.sum()
    
    if signals_count < 10:
        print(f"\n📊 Threshold {threshold}%+: {signals_count} signals (skipped)")
        continue
    
    # Create signals list: (original_index, signal_type, confidence)
    signals = []
    for i, (is_signal, conf) in enumerate(zip(mask, confidence_v13)):
        if is_signal:
            original_idx = v13_test_indices[i]
            signals.append((original_idx, 1, conf))  # 1 = BUY
    
    # Run backtest
    result = run_backtest(
        signals=signals,
        prices_df=test_df,
        initial_balance=INITIAL_BALANCE,
        tp_pips=TP_PIPS,
        sl_pips=SL_PIPS
    )
    
    v13_results[threshold] = result
    
    print(f"\n📊 Threshold {threshold}%+:")
    print(f"   Trades: {result['total_trades']:,}")
    print(f"   Wins: {result['wins']:,} ({result['win_rate']:.1f}%)")
    print(f"   Losses: {result['losses']:,}")
    print(f"   Profit Factor: {result['profit_factor']:.2f}")
    print(f"   Final Balance: ${result['final_balance']:,.2f}")
    print(f"   Net P/L: ${result['profit']:+,.2f}")

# Find best threshold
if v13_results:
    valid_results = {k: v for k, v in v13_results.items() if v['total_trades'] >= 100}
    if valid_results:
        best_v13_th = max(valid_results.keys(), key=lambda x: valid_results[x]['profit_factor'])
        best_v13 = v13_results[best_v13_th]
        
        print("\n" + "-"*80)
        print(f"🎯 V13 BEST: {best_v13_th}%+ threshold")
        print(f"   {best_v13['total_trades']:,} trades, {best_v13['win_rate']:.1f}% win rate")
        print(f"   PF: {best_v13['profit_factor']:.2f}, Final: ${best_v13['final_balance']:,.2f}")


📈 V13 BACKTEST RESULTS (BUY-only)

📊 Threshold 50%+:
   Trades: 30,006
   Wins: 13,720 (45.7%)
   Losses: 16,286
   Profit Factor: 1.26
   Final Balance: $-1,829.00
   Net P/L: $-2,829.00

📊 Threshold 55%+:
   Trades: 21,113
   Wins: 9,621 (45.6%)
   Losses: 11,492
   Profit Factor: 1.26
   Final Balance: $-1,154.90
   Net P/L: $-2,154.90

📊 Threshold 60%+:
   Trades: 8,199
   Wins: 3,875 (47.3%)
   Losses: 4,324
   Profit Factor: 1.34
   Final Balance: $2,521.20
   Net P/L: $+1,521.20

📊 Threshold 65%+:
   Trades: 27
   Wins: 16 (59.3%)
   Losses: 11
   Profit Factor: 2.18
   Final Balance: $1,035.20
   Net P/L: $+35.20

📊 Threshold 70%+: 0 signals (skipped)

📊 Threshold 75%+: 0 signals (skipped)

--------------------------------------------------------------------------------
🎯 V13 BEST: 60%+ threshold
   8,199 trades, 47.3% win rate
   PF: 1.34, Final: $2,521.20


## 📊 Final Comparison: V10 vs V11 vs V13

In [22]:
print("="*80)
print("📊 FINAL MODEL COMPARISON")
print("="*80)

# Collect best results from each version
comparison = []

# V10 best - need to recompute if not available
if 'v10_results' in dir() and v10_results:
    valid_v10 = {k: v for k, v in v10_results.items() if v['total_trades'] >= 100}
    if valid_v10:
        best_v10_th = max(valid_v10.keys(), key=lambda x: valid_v10[x]['profit_factor'])
        best_v10 = v10_results[best_v10_th]
        comparison.append({
            'Version': 'V10',
            'Threshold': f"{best_v10_th}%+",
            'Trades': best_v10['total_trades'],
            'Win Rate': f"{best_v10['win_rate']:.1f}%",
            'Profit Factor': best_v10['profit_factor'],
            'Final Balance': best_v10['final_balance'],
            'Net P/L': best_v10['profit']
        })

# V11 best
if 'v11_results' in dir() and v11_results:
    valid_v11 = {k: v for k, v in v11_results.items() if v['total_trades'] >= 100}
    if valid_v11:
        best_v11_th = max(valid_v11.keys(), key=lambda x: valid_v11[x]['profit_factor'])
        best_v11 = v11_results[best_v11_th]
        comparison.append({
            'Version': 'V11',
            'Threshold': f"{best_v11_th}%+",
            'Trades': best_v11['total_trades'],
            'Win Rate': f"{best_v11['win_rate']:.1f}%",
            'Profit Factor': best_v11['profit_factor'],
            'Final Balance': best_v11['final_balance'],
            'Net P/L': best_v11['profit']
        })

# V13 best
if 'v13_results' in dir() and v13_results:
    valid_v13 = {k: v for k, v in v13_results.items() if v['total_trades'] >= 100}
    if valid_v13:
        best_v13_th = max(valid_v13.keys(), key=lambda x: valid_v13[x]['profit_factor'])
        best_v13 = v13_results[best_v13_th]
        comparison.append({
            'Version': 'V13',
            'Threshold': f"{best_v13_th}%+",
            'Trades': best_v13['total_trades'],
            'Win Rate': f"{best_v13['win_rate']:.1f}%",
            'Profit Factor': best_v13['profit_factor'],
            'Final Balance': best_v13['final_balance'],
            'Net P/L': best_v13['profit']
        })

if comparison:
    comparison_df = pd.DataFrame(comparison)
    print(comparison_df.to_string(index=False))
    
    # Find winner
    winner = max(comparison, key=lambda x: x['Profit Factor'])
    print("\n" + "="*80)
    print(f"🏆 WINNER: {winner['Version']}")
    print(f"   Profit Factor: {winner['Profit Factor']:.2f}")
    print(f"   Win Rate: {winner['Win Rate']}")
    print(f"   Net P/L: ${winner['Net P/L']:+,.2f}")
    print("="*80)
else:
    print("⚠️ No results available for comparison")

📊 FINAL MODEL COMPARISON
Version Threshold  Trades Win Rate  Profit Factor  Final Balance  Net P/L
    V10    0.75%+    6427    52.2%       1.640228         4846.5   3846.5
    V11     0.7%+    6139    51.6%       1.598419         1575.2    575.2
    V13      60%+    8199    47.3%       1.344241         2521.2   1521.2

🏆 WINNER: V10
   Profit Factor: 1.64
   Win Rate: 52.2%
   Net P/L: $+3,846.50
